# Training segmwntation model to find task numbers using Detectron2

# Install detectron2

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('../detectron2'))

In [ ]:
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from IPython import display
import PIL


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


def cv2_imshow(a):
    """A replacement for cv2.imshow() for use in Jupyter notebooks.
    Args:
    a : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
      (N, M, 3) is an NxM BGR color image. shape (N, M, 4) is an NxM BGRA color
      image.
    """
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display.display(PIL.Image.fromarray(a))

# Train on a custom dataset

## Prepare the dataset

In [ ]:
dataset_path=r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\all"

### train dataset

In [ ]:

from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\instances_bollard6.json"
                        ,r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\pollerCroppedDirection")

### evaluatioin dataset

### To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
#visualize training data
train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in dataset_dicts:
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
import matplotlib.pyplot as plt


im = cv2.imread(r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\pollerCroppedDirection\20240825_144317_fixing_5.jpg")


cfg.MODEL.ROI_HEADS.NUM_CLASSES =1

# Create predictor
predictor = DefaultPredictor(cfg)
# Make prediction
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], train_metadata, scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))

## Train!


### ouput directory

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = os.path.join(dataset_path,"output")

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\datasets\big_bollard_dataset"

### train without validation

In [ ]:
cfg= get_cfg()
cfg.OUTPUT_DIR = r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml") 
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.DATASETS.TEST = ("my_dataset_train",)
cfg.SOLVER.BASE_LR =0.00025  
cfg.SOLVER.MAX_ITER = 1000
cfg.TEST.EVAL_PERIOD = 201
cfg.SOLVER.CHECKPOINT_PERIOD = 200
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES =1
cfg.INPUT.MASK_FORMAT = "bitmask"
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### train with validation

In [ ]:
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg= get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_validation",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025 
cfg.SOLVER.MAX_ITER = 10000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES =2
cfg.TEST.EVAL_PERIOD = 500 
cfg.SOLVER.CHECKPOINT_PERIOD = 500
trainer = CocoTrainer(cfg) 

trainer.resume_or_load(resume=False)
trainer.train()

### Continue training from checkpoint


#### with validation


In [ ]:
cfg.merge_from_file(os.path.join(cfg.OUTPUT_DIR,"config15000_29.07.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_validation",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,"model_final.pth")   # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 20000   
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD= 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES =2  
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

#### without validation

In [ ]:
cfg.merge_from_file(os.path.join(cfg.OUTPUT_DIR,"config.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,"model_final.pth")   # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 600   
cfg.SOLVER.CHECKPOINT_PERIOD = 100
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES =2  
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

### save config

In [ ]:
cfg.OUTPUT_DIR = r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output"

In [ ]:
import torch
torch.save(trainer.model.state_dict(),r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output\model_500.pth")

In [ ]:
#save config
with open(os.path.join(cfg.OUTPUT_DIR, "config1000.yaml"), "w") as f:
    f.write(cfg.dump())  # Save as YAML

## TensorBoard

In [4]:
%load_ext tensorboard


In [6]:
%tensorboard --logdir r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output"

# Inference & evaluation using the trained model



## load test dataset

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_test", {}, os.path.join(dataset_path,"test_annotations.json"), 
                     os.path.join(dataset_path,"test")) 
test_metadata = MetadataCatalog.get("my_dataset_test")


In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_test", {},r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\instances_bollard6.json",
                        r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\pollerCroppedDirection")

## load model

In [ ]:
cfg.merge_from_file( os.path.join(cfg.OUTPUT_DIR,"config15000_29.07.yaml"))
cfg.MODEL.WEIGHTS =  os.path.join(cfg.OUTPUT_DIR,"model_0014499.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)


In [5]:
cfg= get_cfg()
cfg.merge_from_file(r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output\config1000.yaml")
cfg.MODEL.WEIGHTS =  os.path.join(r"C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output\model_0000999.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


[10/16 18:57:34 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output\model_0000999.pth ...


## evaluate its performance using AP metric implemented in COCO API.


In [6]:

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[10/16 18:57:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\output\model_0000999.pth ...
[10/16 18:57:36 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[10/16 18:57:36 d2.data.datasets.coco]: Loaded 86 images in COCO format from C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\instances_bollard6.json
[10/16 18:57:36 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    rope    | 124          |
|            |              |
[10/16 18:57:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/16 18:57:36 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[10/16 18:57:

c:\ProgramData\anaconda3\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/16 18:57:46 d2.evaluation.evaluator]: Inference done 11/86. Dataloading: 0.0008 s/iter. Inference: 0.1781 s/iter. Eval: 0.0037 s/iter. Total: 0.1827 s/iter. ETA=0:00:13
[10/16 18:57:51 d2.evaluation.evaluator]: Inference done 37/86. Dataloading: 0.0011 s/iter. Inference: 0.1809 s/iter. Eval: 0.0098 s/iter. Total: 0.1919 s/iter. ETA=0:00:09
[10/16 18:57:56 d2.evaluation.evaluator]: Inference done 63/86. Dataloading: 0.0011 s/iter. Inference: 0.1824 s/iter. Eval: 0.0104 s/iter. Total: 0.1940 s/iter. ETA=0:00:04
[10/16 18:58:01 d2.evaluation.evaluator]: Total inference time: 0:00:16.670323 (0.205806 s / iter per device, on 1 devices)
[10/16 18:58:01 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:14 (0.184131 s / iter per device, on 1 devices)
[10/16 18:58:01 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[10/16 18:58:01 d2.evaluation.coco_evaluation]: Saving results to C:\Users\main laptop\programation\Study\baechlor arbeit\segmentation model\

## evaluate directly

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.MASK_ON = True
cfg.MODEL.MASK_FORMAT = "bitmask"
predictor = DefaultPredictor(cfg)
for imageName in glob.glob(r"C:\Users\main laptop\programation\Study\baechlor arbeit\bollard\images\bollard6\pollerCroppedDirection\*jpg"):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=train_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])